<a href="https://colab.research.google.com/github/kassymoto/Kakaku/blob/master/kakaku_rank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# -*- coding: utf-8 -*-

"""
価格コムのあるカテゴリのランキングのURLを指定すると、
そのランキングの上位N個（多分40個まで）の価格推移のグラフを描画する。
"""

#もろもろのインポート
import requests
from bs4 import BeautifulSoup
from datetime import datetime

#データを格納するためのクラス
class Kakaku:
  url = ""
  price = []
  date = []
  title = ""

#上位何個を表示するか
top = 5

#ランキングのURL
rank_url = requests.get('http://kakaku.com/pc/pc-memory/ranking_0520/spec=101-6/')
soup = BeautifulSoup(rank_url.content, "html.parser")

#ランキングのページから各商品のURLを引っこ抜く
urls = []
num = 0
for i in soup.find_all("a", class_="withIcnLimited"):
  #価格推移グラフのURLは商品のURLにpricehistoryを追加したものである。
    urls.append(str(i.get("href")) + "pricehistory/")
    num+=1
    #指定した個数になったら終了する。
    if num==top:
      break
      
#上記ループで取得したURLは文字列なので、これを使ってrequestsする
url = []
for i in urls:
    url.append(requests.get(i))

#URLをインスタンスに登録していく
kakaku_list = []
for i in url:
  kakaku_tmp = Kakaku()
  kakaku_tmp.url = i
  kakaku_list.append(kakaku_tmp)

rank = 1
for kakaku in kakaku_list:
    soup = BeautifulSoup(kakaku.url.content, "html.parser")
    
    date = soup.find_all("td", class_="alignL")
    price = soup.find_all("td", class_="alignR itemviewColor06")
    
    title = soup.find("title").string.split(" ")[2:-1:]
    kakaku.title = str(rank) + ": " + str(" ".join(title))
    rank+=1
    
    date_tmp = []
    price_tmp = []
    
    for i,j in zip(date[0::2],price):
        if(str(i.string)=="現在"):
            date_tmp.append(datetime.now().strftime('%Y-%m-%d'))
        else:
            date_tmp.append("".join(str(i.string).split(" ")[:-1:]).replace("年","-").replace("月","-").replace("日",""))
        price_tmp.append(int(str(j.string[1::]).replace(',','')))
    
    kakaku.date = date_tmp
    kakaku.price = price_tmp

""" 
for i in kakaku_list:
    print(i.title)
    for j,k in zip(i.date,i.price):
        print(j,k)
    print()
"""

import pandas as pd
kakaku_Title = []
kakaku_Date = []
kakaku_Price = []

for i in kakaku_list:
    for j in range(len(i.date)):
        kakaku_Title.append(i.title)
    kakaku_Date += i.date
    kakaku_Price += i.price

kakaku_data = pd.DataFrame(
        {"Title":kakaku_Title,
         "Date":kakaku_Date,
         "Price":kakaku_Price})

#print(kakaku_data)



import altair as alt

alt.Chart(kakaku_data, height=500 ,width=700).configure_legend(labelLimit=0).mark_line().encode(
        x="Date:T",
        y="Price:Q",
        color=alt.Color('Title', legend=alt.Legend(title="Title", orient='right'))
        )

Chart({
  config: Config({
    legend: LegendConfig({
      labelLimit: 0
    })
  }),
  data:            Date  Price                                              Title
  0    2019-01-21  13980  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  1     2019-1-18  13980  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  2      2019-1-8  14980  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  3      2019-1-8  16799  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  4      2019-1-8  16800  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  5      2019-1-8  18160  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  6      2019-1-7  18468  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  7      2019-1-7  15980  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  8      2019-1-7  15800  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  9      2019-1-7  15026  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  10     2019-1-7  14980  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  11     2019-1-6  14980  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  12     2019-1-3  14980  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  13     2019-1-3  14980  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  14     2019-1-2  14980  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  15   2018-12-16  14980  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  16   2018-12-12  14980  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  17   2018-12-12  16178  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  18    2018-12-9  17798  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  19   2018-11-25  17798  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  20   2018-11-24  17623  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  21   2018-11-11  17798  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  22    2018-11-3  17798  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  23    2018-9-29  18338  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  24     2018-9-8  19980  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  25     2018-9-3  19980  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  26    2018-8-29  22234  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  27    2018-8-29  22237  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  28    2018-8-23  22234  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  29     2018-8-9  22237  1: crucial Ballistix W4U2666BMS-8G [DDR4 PC4-2...
  ..          ...    ...                                                ...
  163   2018-8-12  17980  4: G.Skill F4-2400C15D-16GNT [DDR4 PC4-19200 8...
  164  2019-01-21  12480  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  165   2019-1-18  12480  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  166   2019-1-17  12980  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  167   2019-1-16  13949  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  168   2019-1-16  13950  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  169   2019-1-15  13959  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  170   2019-1-12  13960  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  171   2019-1-12  12980  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  172  2018-12-31  13960  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  173  2018-12-17  13960  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  174  2018-12-17  12980  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  175  2018-12-14  12979  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  176  2018-12-14  12980  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  177  2018-12-12  12979  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  178  2018-12-11  12980  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  179  2018-12-10  13954  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  180   2018-12-7  13955  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  181   2018-12-6  13960  5: ADATA AD4U266638G19-D [DDR4 PC4-21300 8GB 2枚組]
  182   2018-12-3  13968  5: AD